In [ ]:
using Newtonsoft.Json;
var input = new Queue<char>(System.IO.File.ReadAllText("input-sample.txt").ToList());


In [ ]:
public class Packet{
    public int Version{get;set;}
    public int TypeId{get;set;}
    public char? LengthTypeId {get;set;}
    public string BinaryValue{get;set;}
    public string UnusedBinary{get;set;}
    public List<Packet> SubPackets{get;set;}
}

static string BinaryToHex(string binary) => Convert.ToInt32(binary,2).ToString("X");
static string HexToBinary(char hex) => Convert.ToString(Convert.ToInt32(hex.ToString(), 16),2).PadLeft(4,'0');

var packets = new List<Packet>();

In [ ]:
6 < 6

False

In [ ]:
public Packet DecodePacket(string binary){
    while(binary.Length <= 6){
        binary += HexToBinary(input.Dequeue());
    }
    
    var version = int.Parse(BinaryToHex(binary.Substring(0,3)));
    var typeId = int.Parse(BinaryToHex(binary.Substring(3,3)));
    
    binary = binary.Substring(6);
    var lengthTypeId = binary[0];
    
    while(input.Any()){
        binary += HexToBinary(input.Dequeue());
        if(typeId == 4 ){
            var leadDigits = binary.Where((x,i) => i % 5 == 0);
            
            if(leadDigits.Any(x => x == '0')){
                var valueCount = leadDigits.TakeWhile(x => x == '1').Count() + 1;
                if(binary.Length < valueCount * 5) continue;
                binary = binary.Substring(0, valueCount * 5);
                
                return new Packet{
                    Version = version,
                    TypeId = typeId,
                    LengthTypeId = null,
                    BinaryValue = binary.Substring(0, valueCount * 5),
                    UnusedBinary = binary.Substring(valueCount * 5),
                    SubPackets = new List<Packet>()
                };
            }
        }else{
            var subPackets = new List<Packet>();
            if(lengthTypeId == '0' && binary.Length >= 16){
                var bitsCount = Convert.ToInt32(BinaryToHex(binary.Substring(1,15)),16);
                binary = binary.Substring(16);

                while(binary.Length < bitsCount){
                    binary += HexToBinary(input.Dequeue());
                } 

                var subBinary = new string(binary);

                while(!subBinary.All(x => x == '0')){
                    var subPacket = DecodePacket(subBinary);
                    subPackets.Add(subPacket);
                    subBinary = subPacket.UnusedBinary
                }
                binary = binary.Substring(1, (15 + bitsCount));

                return new Packet{
                    Version = version,
                    TypeId = typeId,
                    LengthTypeId = typeId == 4 ? null : lengthTypeId,
                    BinaryValue = binary,
                    UnusedBinary = 
                };
            }else if(lengthTypeId == '1' && binary.Length >= 12){
                var count = Convert.ToInt32(BinaryToHex(binary.Substring(1,11)),16);
                binary = binary.Substring(12);
                
                for(var i = 0; i < count; i++){
                    subPackets.Add(DecodePacket(binary));
                }             
                
                ReachedEnd = true;
                break;
            }
        }
    }
    
    return new Packet{
        Version = version,
        TypeId = typeId,
        LengthTypeId = typeId == 4 ? null : lengthTypeId,
        BinaryValue = binary,
        UnusedBinary = 
    };
}


In [ ]:
Packet packet = DecodePacket();

while(packet != null){
    packets.Add(packet);
    packet = DecodePacket();
}

packets


1000000000100000000000000000010110


index,Version,TypeId,LengthTypeId,BinaryValue
0,3,0,1,000000000100000000000000000010110
1,0,4,<null>,01010
2,3,0,1,1011001000100000000010000100011000111000110100


In [ ]:
packets.Sum(x => x.Version)

6